<font size=5>

Classification with PySpark ML and MLLIB

</font>

<font size=5> This dataset does not have column name, but we will give the proper columns.  Data is downloadable online.  You need to specify the path, either with file:// if it is in the OS file path, if not, the dataset is assumed in an HDFS directory
    
    George Jen  -- Jen Tek LLC
    
</font>

In [2]:
import sys,os,os.path
os.environ['SPARK_HOME']='/home/bigdata2/spark/spark'
%matplotlib inline
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()
#Read Data
df = spark.read.csv("datasets/SMSSpamCollection", sep = "\t", inferSchema=True, header = False)

<font size=5> Show 3 lines to get an idea about the dataset,  _c0 looks like as a label, c1 looks feature </font> 

In [3]:
df.show(3, truncate = False)

+----+-----------------------------------------------------------------------------------------------------------------------------------------------------------+
|_c0 |_c1                                                                                                                                                        |
+----+-----------------------------------------------------------------------------------------------------------------------------------------------------------+
|ham |Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...                                            |
|ham |Ok lar... Joking wif u oni...                                                                                                                              |
|spam|Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's|
+----+----------------

<font size=5> Note: Spam is Spam, Han is OK.  Rename Column name _c0 as status, _c1 as feature  </font>

In [4]:
df = df.withColumnRenamed('_c0', 'status').withColumnRenamed('_c1', 'message')
df.show(3, truncate = False)

+------+-----------------------------------------------------------------------------------------------------------------------------------------------------------+
|status|message                                                                                                                                                    |
+------+-----------------------------------------------------------------------------------------------------------------------------------------------------------+
|ham   |Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...                                            |
|ham   |Ok lar... Joking wif u oni...                                                                                                                              |
|spam  |Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's|
+------+--

<font size=5> 

Encode status column to numeric: ham to 1.0 and spam to 0. All our fields need to be numeric for machine to learn, also rename the column status to label
    
</font>

In [5]:
df.createOrReplaceTempView('temp')
df = spark.sql('select case status when "ham" then 1.0  else 0 end as label, message from temp')
df.show(3, truncate = False)

+-----+-----------------------------------------------------------------------------------------------------------------------------------------------------------+
|label|message                                                                                                                                                    |
+-----+-----------------------------------------------------------------------------------------------------------------------------------------------------------+
|1.0  |Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...                                            |
|1.0  |Ok lar... Joking wif u oni...                                                                                                                              |
|0.0  |Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's|
+-----+---------

<font size=5> 1 is OK, 0 is Junk </font>

<font size=5>
Tokenize the messages
Tokenization is the process of taking text (such as a sentence) and breaking it into individual terms (usually words). Let’s tokenize the messages and create a list of words of each message.
</font>

In [6]:
from pyspark.ml.feature import  Tokenizer
tokenizer = Tokenizer(inputCol="message", outputCol="words")
wordsData = tokenizer.transform(df)
wordsData.show(3, False)

+-----+-----------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|label|message                                                                                                                                                    |words                                                                                                                                                                                   |
+-----+-----------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------

<font size=5> CountVectorizer converts a collection of text documents to vectors of token counts. 
    
See:
https://spark.apache.org/docs/latest/ml-features#countvectorizer


</font>


In [7]:
from pyspark.ml.feature import CountVectorizer
count = CountVectorizer (inputCol="words", outputCol="rawFeatures")
model = count.fit(wordsData)
featurizedData = model.transform(wordsData)
featurizedData.show(3,False)

+-----+-----------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|label|message                                                                                                                                                    |words                                                                                                                                                                                   |rawFeatures                                                                                         

<font size=5>
Apply Term frequency - inverse document frequency (TF-IDF)

#IDF reduces the features that often appear in the corpus. When using text as a feature, this usually improves performance because the most common, and therefore less important, words are weighted down.

</font>

In [8]:
from pyspark.ml.feature import  IDF
idf = IDF(inputCol="rawFeatures", outputCol="features")
idfModel = idf.fit(featurizedData)
rescaledData = idfModel.transform(featurizedData)
rescaledData.select("label", "features").show(3,False)  #Only needed to train


+-----+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|label|features                                                                                                                                                                                                                                                                                                                                                                                                                                                   

<font size=5>
Randomly Split DataFrame into 80% Training (trainDF) and 20 Testing (testDF)
    
</font>


In [9]:
seed = 0  # random seed 0
trainDF, testDF = rescaledData.randomSplit([0.8,0.2],seed)

<font size=5> counts of train and test DataFrame </font>

In [10]:
trainDF.count()

4424

In [11]:
testDF.count()

1150

<font size=5>
Try different classifiers. 

Logistic regression classifier

Logistic regression is a common method of predicting classification responses. A special case of a generalized linear model is the probability of predicting a result. In spark.ml, logistic regression can be used to predict binary results by binomial logistic regression, or it can be used to predict multiple types of results by using multiple logistic regression. Use the family parameter to choose between these two algorithms, or leave it unset and Spark will infer the correct variable.

</font>

In [12]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
import numpy as np
lr = LogisticRegression(maxIter = 100)

model_lr = lr.fit(trainDF)


In [13]:
prediction_lr = model_lr.transform(testDF)

In [14]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
my_eval_lr = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='label', metricName='areaUnderROC')
my_eval_lr.evaluate(prediction_lr)

0.8798701298701298

In [15]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
my_mc_lr = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='label', metricName='f1')
my_mc_lr.evaluate(prediction_lr)

0.9659230678663313

In [16]:
my_mc_lr = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='label', metricName='accuracy')
my_mc_lr.evaluate(prediction_lr)

0.9678260869565217

In [17]:
train_fit_lr = prediction_lr.select('label','prediction')
train_fit_lr.groupBy('label','prediction').count().show()

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|  0.0|       0.0|  117|
|  0.0|       1.0|   37|
|  1.0|       1.0|  996|
+-----+----------+-----+



<font size=5>
Naive Bayes 
Naive Bayesian classifiers are a class of simple probability classifiers that apply strong (naive) independent assumptions between features based on Bayes' theorem. The spark.ml implementation currently supports polynomial naive Bayes and Bernoulli Naïve Bayes.
</font>

In [18]:
from pyspark.ml.classification import NaiveBayes
nb = NaiveBayes()
Model_nb = nb.fit(trainDF)

In [19]:
predictions_nb = Model_nb.transform(testDF)
predictions_nb.select('label', 'prediction').show(5)

+-----+----------+
|label|prediction|
+-----+----------+
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
+-----+----------+
only showing top 5 rows



In [20]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
my_eval_nb = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='label', metricName='areaUnderROC')
my_eval_nb.evaluate(predictions_nb)

0.949864392635477

In [21]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
my_mc_nb = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='label', metricName='f1')
my_mc_nb.evaluate(predictions_nb)

0.9372189798267707

In [22]:
my_mc_nb = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='label', metricName='accuracy')
my_mc_nb.evaluate(predictions_nb)

0.9321739130434783

<font size=5>

Now let's try Random Forest Classfication to see how it performs on the classification on the same data
    
    
</font>

In [23]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import IndexToString, StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator


In [24]:
rf = RandomForestClassifier(featuresCol='features', labelCol='label', predictionCol='prediction', probabilityCol='probability', rawPredictionCol='rawPrediction',maxDepth=3)
Model_rf = rf.fit(trainDF)

In [25]:
predictions_rf = Model_rf.transform(testDF)
predictions_rf.select('label', 'prediction').show(5)

+-----+----------+
|label|prediction|
+-----+----------+
|  0.0|       1.0|
|  0.0|       1.0|
|  0.0|       1.0|
|  0.0|       1.0|
|  0.0|       1.0|
+-----+----------+
only showing top 5 rows



In [26]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
my_eval_rf = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='label', metricName='areaUnderROC')
my_eval_rf.evaluate(predictions_rf)

0.5

<font size=5>

Given Area Under Curve is 0.5, we do not want to use this Random Forest Classification.  Area under Curve is between 0 to 1,
the more close to 1 the better the classication is.

We wil give up Random Forest on this classitication

</font>

<font size=5>

There are 2 machine learning libraries from Apache Spark.

pyspark.ml and pyspark.mllib.  Data for pyspark.ml is in the form of DataFrame, but in pyspark.mllib, data is requried in the form of RDD (Resilient Distributed Datasets), on top of that, pyspark.mllib training and testing data are required to be in the format of labeled point. See definition

https://spark.apache.org/In fact, it is lot harder to prepare data for machine learning in pyspark.mllib than in pyspark.ml.

docs/2.1.0/mllib-data-types.html#labeled-point


In fact, it is lot harder to prepare data for machine learning in pyspark.mllib than in pyspark.ml.

Now we switch from pyspark.ml to pyspark.mllib.
    
    
    
</font>

In [27]:
from pyspark.mllib.linalg import SparseVector
from pyspark.mllib.regression import LabeledPoint
from pyspark.ml.linalg import Vector as MLVector, Vectors as MLVectors
from pyspark.mllib.linalg import Vector as MLLibVector, Vectors as MLLibVectors



<font size=5>

Below function takes a line from RDD and returns line that complies with LabeledPoint specification.
    
</font>

In [30]:
def parsePoint(line):
    return LabeledPoint(line[0], MLLibVectors.fromML(line[1]))

In [33]:
trainDF.show(5,False)

+-----+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [31]:

trainDF_2=trainDF.selectExpr("label as first",'features')
trainDF_2.columns
    

['first', 'features']

<font size=5>

Convert a training DataFrame into RDD


</font>

In [35]:
trainDF_rdd=trainDF_2.rdd

<font size=5>

Resulting RDD undergoes mapped to parsePoint function, converting into RDD with each line meeting the requirement of Labeled Point, which is needed to be used as training and testing data when calling train and predict


</font>

In [36]:
trainDF_lp=trainDF_rdd.map(parsePoint)

<font size=5>

Train the SVMwithSGD model using RDD that complies with Labeled Point

</font>

In [38]:
from pyspark.mllib.classification import SVMWithSGD, SVMModel
from pyspark.mllib.regression import LabeledPoint


model_mllib_svm = SVMWithSGD.train(trainDF_lp, iterations=100)


<font size=5>
    
Evaluate SVM model using training data

</font>

In [39]:
# Evaluating the model on training data
labelsAndPreds = trainDF_lp.map(lambda p: (p.label, model_mllib_svm.predict(p.features)))
trainErr = labelsAndPreds.filter(lambda lp: lp[0] != lp[1]).count() / float(trainDF_lp.count())
print("Training Error = " + str(trainErr))


Training Error = 0.0038426763110307413


<font size=5>

Evaluate SVM model using testing data, which also needs to be converted to Labeled Point format.

</font>

In [40]:
testDF_2=testDF.selectExpr("label as first",'features')
testDF_2.columns

['first', 'features']

In [41]:
testDF_rdd=testDF_2.rdd


In [42]:
testDF_lp=testDF_rdd.map(parsePoint)

In [43]:
# Evaluating the model on testing data
labelsAndPreds = testDF_lp.map(lambda p: (p.label, model_mllib_svm.predict(p.features)))
testErr = labelsAndPreds.filter(lambda lp: lp[0] != lp[1]).count() / float(testDF_lp.count())
print("Testing Error = " + str(testErr))


Testing Error = 0.030434782608695653


<font size=5>
    
Now get additional testing metrics with BinaryClassificationMetrics, on 

Area Under Precision Recall and Area under ROC Curve.

Persoanlly, I look more on Area under ROC Curve to evaluate model performance, which is more reliable.



</font>

In [44]:
from pyspark.mllib.evaluation import BinaryClassificationMetrics

In [45]:
predictionAndLabels = testDF_lp.map(lambda lp: (float(model_mllib_svm.predict(lp.features)), lp.label))

In [46]:
# Instantiate metrics object
metrics = BinaryClassificationMetrics(predictionAndLabels)

# Area under precision-recall curve
print("Area under PR = %s" % metrics.areaUnderPR)

# Area under ROC curve
print("Area under ROC = %s" % metrics.areaUnderROC)

Area under PR = 0.9764907668322788
Area under ROC = 0.9193005789391331


<font size=5>

Since at it, try LogisticRegressionWithLBFGS under pyspark.mllib with the same way on SVM above.

    
</font>

In [47]:
from pyspark.mllib.classification import LogisticRegressionWithLBFGS, LogisticRegressionModel


In [48]:
# Build the model
model_mllib_lr_BFGS = LogisticRegressionWithLBFGS.train(trainDF_lp)


In [49]:
# Evaluating the model on training data
labelsAndPreds = trainDF_lp.map(lambda p: (p.label, model_mllib_lr_BFGS.predict(p.features)))
trainErr = labelsAndPreds.filter(lambda lp: lp[0] != lp[1]).count() / float(trainDF_lp.count())
print("Training Error = " + str(trainErr))

Training Error = 0.0


In [50]:
# Evaluating the model on testing data
labelsAndPreds = testDF_lp.map(lambda p: (p.label, model_mllib_lr_BFGS.predict(p.features)))
testErr = labelsAndPreds.filter(lambda lp: lp[0] != lp[1]).count() / float(testDF_lp.count())
print("Testing Error = " + str(testErr))

Testing Error = 0.03739130434782609


In [51]:
predictionAndLabels = testDF_lp.map(lambda lp: (float(model_mllib_lr_BFGS.predict(lp.features)), lp.label))

In [52]:
# Instantiate metrics object
metrics = BinaryClassificationMetrics(predictionAndLabels)

# Area under precision-recall curve
print("Area under PR = %s" % metrics.areaUnderPR)

# Area under ROC curve
print("Area under ROC = %s" % metrics.areaUnderROC)

Area under PR = 0.9671056286416305
Area under ROC = 0.8878370625358576


<font size=5>

This concludes pyspark ML/MLLIIB classification Excercise.

</font>